In [1]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_recall_curve

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from skl_plus.bayes_models.discriminant_analysis import LDAClassifier, QDAClassifier 

# Comparision on Synthetic Data

## Generating Synthetic Data

In [5]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [6]:
syn_X, syn_y = make_classification(n_samples=1000, n_features=10,
                                  n_classes=3, n_informative=7,
                                  weights=[0.3, 0.5, 0.2], random_state=42)

In [7]:
syn_X_train, syn_X_test, syn_y_train, syn_y_test = train_test_split(syn_X, syn_y, train_size=0.8, 
                                                                    stratify=syn_y, random_state=42)

## LDA Comparision

### Training SKL-Plus LDA Implementation

In [8]:
sk_lda = LDAClassifier()
sk_lda.fit(syn_X_train, syn_y_train)

Reloaded
(800, 10)


In [9]:
sk_lda_preds = sk_lda.predict(syn_X_test)

In [10]:
print(classification_report(syn_y_test, sk_lda_preds))

              precision    recall  f1-score   support

           0       0.86      0.98      0.91        60
           1       0.94      0.75      0.83       100
           2       0.61      0.78      0.68        40

    accuracy                           0.82       200
   macro avg       0.80      0.84      0.81       200
weighted avg       0.85      0.82      0.83       200



In [11]:
skl_cross_tab = pd.crosstab(syn_y_test, sk_lda_preds)
skl_cross_tab['total'] = np.sum(skl_cross_tab[0:], axis=1)
skl_cross_tab.loc['total'] = np.sum(skl_cross_tab, axis=0)

In [12]:
skl_cross_tab

col_0,0,1,2,total
row_0,,,,
0,59,0,1,60
1,6,75,19,100
2,4,5,31,40
total,69,80,51,200


In above cross-table, col_0 represents the predicted values for the target variable, sk_lda_preds, and rows_0 represents th real values, syn_y_test.

- Class 0:
    - Out of 60 real values, the model correctly classified 59:
        * Recall -> 59/60 = 98&
    - The model predicted 69 samples as 0, of which 59 were correct:
        * Precision -> 59/69 = 85%

- Class 1:
    - Out of 100 real values, the model correctly classified 75:
        * Recall -> 75/100 = 75%
    - The model predicted 80 samples as 1, of which 75 were correct:
        * Precision -> 75/80 = 94%

- Class 2:
    - Out of 40 real values, the model correctly classified 31:
        * Recall -> 31/40 =  77%
    - The model predicted 51 samples as class 2, of which 31 were correct:
        * Precision ->  31/51 = 60%.


- Training Data Class counts:
    * Class 0 : 240 samples
    * Class 1 : 399 samples
    * Class 2 : 161 samples


- Testing Data Class counts:
    * Class 0 : 60 samples
    * Class 1 : 100 samples
    * Class 2 : 40 samples


We obsereve that both training and testing data is imbalanced, espescially with Class 2 having less samples in training data in comparision to other class. This imbalance likely contributes to the model’s lower recall for Class 2, as it had fewer examples to learn from during training. Additionally, Class 1, having the highest number of samples in the training set, helps explain why the model achieved higher recall and precision for that class.

### SKLearn LDA Model

In [13]:
lda = LinearDiscriminantAnalysis()
lda.fit(syn_X_train, syn_y_train)

LinearDiscriminantAnalysis()

In [14]:
lda_preds = lda.predict(syn_X_test)

In [15]:
print(classification_report(syn_y_test, lda_preds))

              precision    recall  f1-score   support

           0       0.86      0.98      0.91        60
           1       0.86      0.86      0.86       100
           2       0.68      0.53      0.59        40

    accuracy                           0.83       200
   macro avg       0.80      0.79      0.79       200
weighted avg       0.82      0.83      0.82       200



In [16]:
sk_cross_tab = pd.crosstab(syn_y_test, lda_preds)
sk_cross_tab['total'] = np.sum(sk_cross_tab[0:], axis=1)
sk_cross_tab.loc['total'] = np.sum(sk_cross_tab, axis=0)

In [17]:
sk_cross_tab

col_0,0,1,2,total
row_0,,,,
0,59,0,1,60
1,5,86,9,100
2,5,14,21,40
total,69,100,31,200


In above cross-table, col_0 represents the predicted values for the target variable, lda_preds, and rows_0 represents th real values, syn_y_test.

- Class 0:
    - Out of 60 real values, the model correctly classified 59:
        * Recall -> 59/60 = 98%
    - The model predicted 69 samples as 0, of which 59 were correct:
        * Precision -> 59/69 = 85%

- Class 1:
    - Out of 100 real values, the model correctly classified 86:
        * Recall -> 86/100 = 86%
    - The model predicted 100 samples as 1, of which 86 were correct:
        * Precision -> 86/100 = 86%

- Class 2:
    - Out of 40 real values, the model correctly classified 21:
        * Recall -> 21/40 =  53%
    - The model predicted 31 samples as class 2, of which 21 were correct:
        * Precision ->  21/31 = 68%.


### Comparision

Due to the imbalanced class distribution in the dataset, both my implementation of the LDAClassifier and sklearn's LinearDiscriminantAnalysis struggled with Class 2. For Class 2, sklearn achieved a recall of 53%, which is lower than my implementation's recall of 77%, but it had higher precision at 68% compared to my model’s 60%.

Looking at the F1 scores of both models:

**My implementation (LDAClassifier)**:

* Class 0: 91%
* Class 1: 83%
* Class 2: 68%

**Sklearn LDA**:

* Class 0: 91%
* Class 1: 86%
* Class 2: 59%

Both models achieved similar overall accuracy, with my model at 82% and sklearn’s model at 83%, showing no significant difference in overall performance.

For Class 0, both models performed identically.

For Class 1, which has a higher representation in the training set, sklearn’s model predicted 100 test samples as Class 1, showing higher recall (86%) but at the cost of precision (86%). In contrast, my model predicted 85 samples as Class 1, with a higher precision of 94% and a recall of 75%. This suggests that sklearn's model is more confident in classifying samples as Class 1, which might be beneficial for certain applications, but my model's higher precision would be more useful in scenarios where false positives need to be minimized, such as in medical diagnoses.

For Class 2, both models struggled, likely due to the smaller number of samples in the training set. However, my model showed slightly better recall for Class 2, while sklearn's model had higher precision.

In conclusion, both models performed well, with sklearn's LDA likely benefiting from under-the-hood optimizations and more sophisticated techniques. While my implementation using more traditional discriminant function yielded competitive results, especially in terms of precision, sklearn’s model showed better generalization, particularly for Class 1.

## QDA Comparision

### Training SKL-Plus QDA Implementation

In [18]:
sk_qda = QDAClassifier()
sk_qda.fit(syn_X_train, syn_y_train)

(800, 10)


In [19]:
sk_qda_preds = sk_qda.predict(syn_X_test)

In [20]:
print(classification_report(syn_y_test, sk_qda_preds))

              precision    recall  f1-score   support

           0       0.92      0.98      0.95        60
           1       0.92      0.96      0.94       100
           2       0.91      0.72      0.81        40

    accuracy                           0.92       200
   macro avg       0.92      0.89      0.90       200
weighted avg       0.92      0.92      0.92       200



In [21]:
skl_qda_cross_tab = pd.crosstab(syn_y_test, sk_qda_preds)
skl_qda_cross_tab['total'] = np.sum(skl_qda_cross_tab[0:], axis=1)
skl_qda_cross_tab.loc['total'] = np.sum(skl_qda_cross_tab, axis=0)

In [22]:
skl_qda_cross_tab

col_0,0,1,2,total
row_0,,,,
0,59,0,1,60
1,2,96,2,100
2,3,8,29,40
total,64,104,32,200


In above cross-table, col_0 represents the predicted values for the target variable, sk_qda_preds, and rows_0 represents thw real values, syn_y_test.

- Class 0:
    - Out of 60 real values, the model correctly classified 59:
        * Recall -> 59/60 = 98%
    - The model predicted 64 samples as 0, of which 59 were correct:
        * Precision -> 59/64 = 92%

- Class 1:
    - Out of 100 real values, the model correctly classified 96:
        * Recall -> 96/100 = 96%
    - The model predicted 104 samples as 1, of which 96 were correct:
        * Precision -> 96/104 = 92%

- Class 2:
    - Out of 40 real values, the model correctly classified 29:
        * Recall -> 29/40 = 72.5%
    - The model predicted 32 samples as class 2, of which 29 were correct:
        * Precision ->  29/32 = 90%.


### SKLearn QDA Model

In [23]:
qda = QuadraticDiscriminantAnalysis(reg_param=0.007)
qda.fit(syn_X_train, syn_y_train)

QuadraticDiscriminantAnalysis(reg_param=0.007)

In [24]:
qda_preds = qda.predict(syn_X_test)

In [25]:
print(classification_report(syn_y_test, qda_preds))

              precision    recall  f1-score   support

           0       0.92      0.98      0.95        60
           1       0.92      0.96      0.94       100
           2       0.91      0.72      0.81        40

    accuracy                           0.92       200
   macro avg       0.92      0.89      0.90       200
weighted avg       0.92      0.92      0.92       200



In [26]:
sk_qda_cross_tab = pd.crosstab(syn_y_test, qda_preds)
sk_qda_cross_tab['total'] = np.sum(sk_qda_cross_tab[0:], axis=1)
sk_qda_cross_tab.loc['total'] = np.sum(sk_qda_cross_tab, axis=0)

In [27]:
sk_qda_cross_tab

col_0,0,1,2,total
row_0,,,,
0,59,0,1,60
1,2,96,2,100
2,3,8,29,40
total,64,104,32,200


In above cross-table, col_0 represents the predicted values for the target variable, lda_preds, and rows_0 represents th real values, syn_y_test.

- Class 0:
    - Out of 60 real values, the model correctly classified 59:
        * Recall -> 59/60 = 98%
    - The model predicted 69 samples as 0, of which 59 were correct:
        * Precision -> 59/69 = 85%

- Class 1:
    - Out of 100 real values, the model correctly classified 86:
        * Recall -> 86/100 = 86%
    - The model predicted 100 samples as 1, of which 86 were correct:
        * Precision -> 86/100 = 86%

- Class 2:
    - Out of 40 real values, the model correctly classified 21:
        * Recall -> 21/40 =  53%
    - The model predicted 31 samples as class 2, of which 21 were correct:
        * Precision ->  21/31 = 68%.


In above cross-table, col_0 represents the predicted values for the target variable, qda_preds, and rows_0 represents thw real values, syn_y_test.

- Class 0:
    - Out of 60 real values, the model correctly classified 59:
        * Recall -> 59/60 = 98%
    - The model predicted 64 samples as 0, of which 59 were correct:
        * Precision -> 59/64 = 92%

- Class 1:
    - Out of 100 real values, the model correctly classified 96:
        * Recall -> 96/100 = 96%
    - The model predicted 104 samples as 1, of which 96 were correct:
        * Precision -> 96/104 = 92%

- Class 2:
    - Out of 40 real values, the model correctly classified 29:
        * Recall -> 29/40 = 72.5%
    - The model predicted 32 samples as class 2, of which 29 were correct:
        * Precision ->  29/32 = 90%.


### Comparision

As seen here, both sklearn's QuadraticDiscriminantAnalysis with reg_param=0.007 and my own QDA implementation produce exactly identical classification results on the synthetic dataset.
This match confirms that the underlying covariance estimation and discriminant function in my implementation is consistent with sklearn’s, differing only by a small amount of numerical regularization.

- Note: When training sklearn’s QDA model without setting reg_param, it issued warnings indicating that the covariance matrices for multiple classes were not full rank — suggesting multicollinearity in the features. Setting reg_param=0.007 eliminated these warnings and produced stable results identical to my implementation.